<a href="https://colab.research.google.com/github/JefNtungila/Hostels-in-SEA/blob/master/Hostels_in_the_Philippines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hostels in the Philippines

In [0]:
from bs4 import BeautifulSoup
import requests

##Loading in the Cities

In [0]:
country = 'Philippines'

In [0]:


result = requests.get(f"https://www.hostelworld.com/hostels/{country}")

In [0]:
src = result.content
soup = BeautifulSoup(src, 'lxml')
soup.prettify();

In [25]:
locations = ['topcities', 'otherlocations']


list_of_cities = []

for location in locations:
  top_cities_finder = soup.find_all('div', class_=(f"{location}")) #find html classes
  list_of_cities.append(top_cities_finder)

#for every item in the sublist and for every sublist in the list of cities - get all the items
flat_list_of_cities = [item for sublist in list_of_cities for item in sublist]


cities = []

for city in flat_list_of_cities:
  city = city.find_all('a') #find articles in html classes
  cities.append(city)
  
final_flat_list_of_cities = [item for sublist in cities for item in sublist]

city_names = []

for city in final_flat_list_of_cities:
  city_name = city.text
  city_names.append(city_name)

print( len(list_of_cities), len(flat_list_of_cities) , len(cities) , len(final_flat_list_of_cities), len(city_names))

2 3 3 40 40


In [20]:
city_names

['Cebu',
 'Manila',
 'El Nido',
 'Boracay Island',
 'Coron',
 'Puerto Princesa City',
 'Siargao Island',
 'Panglao Island',
 'Malapascua Island',
 'Bohol Island',
 'Bohol',
 'Palawan',
 'Baguio',
 'Baler',
 'Banaue',
 'Batuan',
 'Bohol Island',
 'Boracay Island',
 'Carabao Island',
 'Cebu',
 'Coron',
 'Davao City',
 'Dumaguete',
 'El Nido',
 'Kalibo',
 'Legazpi City',
 'Malapascua Island',
 'Manila',
 'Mati City',
 'Panglao Island',
 'Puerto Galera',
 'Puerto Princesa City',
 'San Felipe',
 'San Jose Negros Oriental',
 'San Juan La Union',
 'San Vicente',
 'Siargao Island',
 'Siquijor',
 'Tablas Island',
 'Tagaytay']

##Getting hostel names

In [0]:
city = 'Cebu'

result1 = requests.get(f"https://www.hostelworld.com/findabed.php/ChosenCity.{city}/ChosenCountry.{country}")
src1 = result1.content
soup1 = BeautifulSoup(src1, 'lxml')
soup1.prettify();




In [22]:
hostel_container = soup1.find_all('div', class_ = 'fabresult rounded clearfix hwta-property')

hostel_names = []

for item in hostel_container:
  item = item.find('a').text
  hostel_names.append(item)
  
  
hostel_names  
    

['Noordzee Hostel',
 'HappyNest Hostel',
 'Murals Hostel and Cafe',
 'Teofel Hostel',
 'Sharky Hostel Oslob',
 'Mad Monkey Cebu City',
 'Chief Mau Moalboal Cebu',
 'Moalboal Backpacker Lodge',
 'Casa Bonita Inn Oslob',
 'Le Village Guesthouse Hostel & Bar',
 'Le Village Hostel Moalboal',
 "Jiji's Hostel",
 'MoHo - Moalboal Hostel',
 'Ananas Guesthouse Moalboal',
 'West Coast Beach House',
 'The Stopover Hostel - Mactan Cebu',
 'J & C Guesthouse',
 'Shejoje Poshtel Hostel',
 "Angler's Hub and Resort",
 'Cebu Guest Inn',
 "Allson's Inn",
 'Bugoybikers in Cebu City',
 'Big Paul Hostel Cebu City',
 '1521 Hostel Mactan Cebu',
 'D522 at Kiener Hills - Hotel Near Cebu Airport',
 "JJ's Place Moalboal",
 'Sunday Hostel',
 "Emok's Guesthouse",
 'Ayo Homtel',
 'Murals Mactan']